In [ ]:
import math
import os
import numpy as np
from sklearn import metrics
from tqdm.notebook import tqdm

import torch
from torch.nn import BCELoss
from torch.optim import RMSprop

import syft as sy
from syft.federated.floptimizer import Optims

from utils import Config, build_model, Metric, Standardizer, EarlyStopping

In [ ]:
hook = sy.TorchHook(torch)

In [ ]:
data_folder = './data/mimic3_17f_24h/'
data_filename = os.path.join(data_folder, 'imputed-normed-ep_1_24.npz')
folds_filename = os.path.join(data_folder, '5-folds.npz')
features_filename = os.path.join(data_folder, 'input.csv')
results_folder = './results/mimic3_17f_24h/'

In [ ]:
if not os.path.exists(data_folder):
    print(f'Wrong data_folder specified. This folder must exist')
    exit(1)

if not os.path.exists(results_folder):
    os.makedirs(results_folder)

In [ ]:
config = Config()
config

In [ ]:
folds_file = np.load(folds_filename, allow_pickle=True)
folds = folds_file['folds_ep_mor'][config.label_type][0]

data_file = np.load(data_filename, allow_pickle=True)
y = data_file['adm_labels_all'][:, config.label_type]
y = (y > 0).astype(float)

X = np.genfromtxt(features_filename, delimiter=',')

In [ ]:
TASK_NAME = 'Mortality'
CLF_NAME = 'FederatedFeedForwardNetwork'

In [ ]:
metrics = [
    Metric('Accuracy', metrics.accuracy_score, use_soft=False),
    Metric('Precision', metrics.precision_score, use_soft=False),
    Metric('Recall', metrics.recall_score, use_soft=False),
    Metric('F1 score', metrics.f1_score, use_soft=False),
    Metric('ROC AUC', metrics.roc_auc_score, use_soft=True),
    Metric('Average precision', metrics.average_precision_score, use_soft=True),
]

In [ ]:
def create_workers(i):
    bob = sy.VirtualWorker(hook, id=f"bob{i}")
    alice = sy.VirtualWorker(hook, id=f"alice{i}")
    james = sy.VirtualWorker(hook, id=f"james{i}")
    return [bob, alice, james]

In [ ]:
def distribute_dataset(X, y, train_idx, test_idx, workers):
    tensor_X, tensor_y = torch.Tensor(X), torch.Tensor(y).view(-1, 1)

    num_train = len(train_idx)
    split = int(np.floor(config.validation_split * num_train))
    train_idx, valid_idx = train_idx[split:], train_idx[:split]
    indices = [train_idx, valid_idx, test_idx]
    tags = ['train', 'valid', 'test']
    
    for idx, tag in zip(indices, tags):
        split_X = torch.split(tensor_X[idx], math.ceil(len(tensor_X[idx]) / len(workers)), dim=0)
        split_y = torch.split(tensor_y[idx], math.ceil(len(tensor_y[idx]) / len(workers)), dim=0)

        for i in range(len(workers)):
            tag_X = split_X[i].tag("#X", f"#{tag}").describe("")
            tag_y = split_y[i].tag("#Y", f"#{tag}").describe("")
        
            tag_X.send(workers[i], garbage_collect_data=False)
            tag_y.send(workers[i], garbage_collect_data=False)
    
    return sy.PrivateGridNetwork(*workers)

In [ ]:
def collect_datasets(grid):
    loaders = []
    tags = ['train', 'valid', 'test']
    for tag in tags:
        found_X = grid.search("#X", f"#{tag}")
        found_y = grid.search("#Y", f"#{tag}")
    
        datasets = []
        for worker in found_X.keys():
            datasets.append(sy.BaseDataset(found_X[worker][0], found_y[worker][0]))

        dataset = sy.FederatedDataset(datasets)
        loaders.append(sy.FederatedDataLoader(dataset, batch_size=config.batch_size))
    
    return loaders

In [ ]:
def train(model, train_loader, valid_loader, workers):
    criterion = BCELoss() # binary cross-entropy
    # for RMSprop in PySyft each worker needs its own optimizer
    worker_ids = [worker.id for worker in workers]
    optims = Optims(worker_ids, optim=RMSprop(model.parameters(), lr=config.learning_rate))
    early_stopping = EarlyStopping(patience=config.early_stopping_patience)
    
    for epoch in tqdm(range(config.epochs)):
        
        model.train()
        for data, target in train_loader:
            model.send(data.location)
            
            opt = optims.get_optim(data.location.id)
            opt.zero_grad()
            
            output = model(data)
            
            loss = criterion(output, target)
            loss.backward()

            opt.step()
            model.get()

        model.eval() 
        valid_losses = []
        for data, target in valid_loader:
            model.send(data.location)
            
            output = model(data)
            loss = criterion(output, target)
            valid_losses.append(loss.get().item())
            
            model.get()
        valid_loss = np.average(valid_losses)
        
        if early_stopping.should_early_stop(valid_loss, model):
            break
    
    model.load_state_dict(early_stopping.best_model_state)
    
    return model

In [ ]:
def predict(model, data_loader):
    model.eval()
    
    num_elements = sum([len(data) for data, _ in data_loader])
    
    predictions = torch.zeros(num_elements)
    targets = torch.zeros(num_elements)
    
    start = 0
    for data, target in data_loader:
        
        target = target.get().view(-1)
        end = start + len(target)
        
        targets[start:end] = target
        
        model.send(data.location)
        with torch.no_grad():
            output = model(data)
            predictions[start:end] = output.get().view(-1)
        model.get()
        start = end
    return predictions, targets

In [ ]:
for i, (train_idx, valid_idx, test_idx) in enumerate(folds):
    train_idx = np.concatenate((train_idx, valid_idx))
    
    standardizer = Standardizer()
    standardizer.fit(X[train_idx])
    X_transformed = standardizer.transform(X)
    
    workers = create_workers(i)
    
    grid = distribute_dataset(X_transformed, y, train_idx, test_idx, workers)
    
    train_loader, valid_loader, test_loader = collect_datasets(grid)

    model = build_model(config, n_features=X_transformed.shape[1])    
    model = train(model, train_loader, valid_loader, workers)
    
    y_soft, y_true = predict(model, test_loader)
    y_pred = (y_soft > 0.5).type(torch.int)

    for metric in metrics:
        if metric.use_soft:
            score = metric.function(y_true, y_soft)
        else:
            score = metric.function(y_true, y_pred)
        metric.scores.append(score)
    
    del train_loader, valid_loader, test_loader, workers, grid

In [ ]:
results_filename = os.path.join(results_folder, f'{TASK_NAME}_{CLF_NAME}.txt')

In [ ]:
with open(results_filename, 'w') as f:
    f.write(f'{TASK_NAME} {CLF_NAME}\n\n')
    f.write(f'{str(config)}\n\n')
    for metric in metrics:
        mean, std = np.mean(metric.scores), np.std(metric.scores)
        print(f'{metric.name}: {mean:.5f} ± {std:.5f}')
        f.write(f'{metric.name}: {mean:.5f} ± {std:.5f}\n')